<a href="https://colab.research.google.com/github/evan-randall/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Evan_Grinalds_LS_DS17_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [35]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.5)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.5)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.5)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.5))]

In [36]:
print(df.shape)
df.head(3)

(48066, 34)


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [37]:
df['created']

0        2016-06-24 07:54:24
1        2016-06-12 12:19:27
2        2016-04-17 03:26:41
3        2016-04-18 02:22:02
4        2016-04-28 01:32:41
                ...         
49346    2016-04-22 15:44:11
49347    2016-06-02 05:41:05
49348    2016-04-04 18:22:34
49349    2016-04-16 02:13:40
49350    2016-04-08 02:13:33
Name: created, Length: 48066, dtype: object

In [38]:
# Format date and time

df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)
df['created']

0       2016-06-24 07:54:24
1       2016-06-12 12:19:27
2       2016-04-17 03:26:41
3       2016-04-18 02:22:02
4       2016-04-28 01:32:41
                ...        
49346   2016-04-22 15:44:11
49347   2016-06-02 05:41:05
49348   2016-04-04 18:22:34
49349   2016-04-16 02:13:40
49350   2016-04-08 02:13:33
Name: created, Length: 48066, dtype: datetime64[ns]

In [39]:
train =  df[(df['created'] >= '2016-04-01') & (df['created'] <= '2016-05-31')]
test = df[(df['created'] >= '2016-06-01') & (df['created'] <= '2016-06-30')]

train.shape, test.shape

((31352, 34), (16709, 34))

 ### Engineer at least two new features.

In [40]:
features = ['bedrooms', 'bathrooms']
target = ['price']

X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]

### Fit a linear regression model with at least two features.

In [41]:
# Fit a linear regression model with at least two features.

from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Get the model's coefficients and intercept.

In [42]:
model.coef_, model.intercept_

(array([[ 392.56988015, 2076.17229508]]), array([497.70168055]))

### Get regression metrics RMSE, MAE, and  𝑅2 , for both the train and test data.

In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Setting the y prediction for train
y_pred_train = model.predict(X_train)

# Calculating the mean squared error
mse_train = mean_squared_error(y_train, y_pred_train)

# Calculating
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print(f'RMSE for train is', f'${rmse_train:.2f}')
print(f'MAE for train is', f'${mae_train:.2f}')
print(f'R^2 for train is', f'${r2_train:.2f}')

RMSE for train is $1225.32
MAE for train is $812.02
R^2 for train is $0.52


In [44]:
# Setting the y prediction for test
y_pred_test = model.predict(X_test)

# Calculating the mean squared error
mse_test = mean_squared_error(y_test, y_pred_test)

# Calculating
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print(f'RMSE for test is', f'${rmse_test:.2f}')
print(f'MAE for test is', f'${mae_test:.2f}')
print(f'R^2 for test is', f'${r2_test:.2f}')

RMSE for test is $1211.50
MAE for test is $818.55
R^2 for test is $0.53


### What's the best test MAE you can get? Share your score and features used with your cohort on Slack!

In [45]:
print(f'MAE for test is', f'${mae_test:.2f}')

MAE for test is $818.55


### Stretch Goals


In [46]:
from sklearn.preprocessing import LabelEncoder

In [47]:
le_interest = LabelEncoder()

In [48]:
df['interest_level'] = le_interest.fit_transform(df['interest_level'])

In [49]:
df.isnull().sum()

bathrooms                  0
bedrooms                   0
created                    0
description             1395
display_address          131
latitude                   0
longitude                  0
price                      0
street_address            10
interest_level             0
elevator                   0
cats_allowed               0
hardwood_floors            0
dogs_allowed               0
doorman                    0
dishwasher                 0
no_fee                     0
laundry_in_building        0
fitness_center             0
pre-war                    0
laundry_in_unit            0
roof_deck                  0
outdoor_space              0
dining_room                0
high_speed_internet        0
balcony                    0
swimming_pool              0
new_construction           0
terrace                    0
exclusive                  0
loft                       0
garden_patio               0
wheelchair_access          0
common_outdoor_space       0
dtype: int64

In [50]:
pet_friendly = ((df.cats_allowed == 1) & (df.dogs_allowed ==1))

In [51]:
df['pet_friendly'] = (df.cats_allowed + df.dogs_allowed)

In [52]:
df.pet_friendly.value_counts()

0    25009
2    21513
1     1544
Name: pet_friendly, dtype: int64

In [53]:
df['pet_friend'] = df['pet_friendly'] == 2

In [54]:
df = df.drop(['pet_friendly'],axis=1)

In [57]:
df.head(1)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pet_friend
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False


In [58]:
list(df)

['bathrooms',
 'bedrooms',
 'created',
 'description',
 'display_address',
 'latitude',
 'longitude',
 'price',
 'street_address',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space',
 'pet_friend']

In [59]:
le_pet = LabelEncoder()
df['pet_friend'] = le_pet.fit_transform(df['pet_friend'])

In [60]:
df['total_rooms'] = (df.bedrooms + df.bathrooms)

In [61]:
train =  df[(df['created'] >= '2016-04-01') & (df['created'] <= '2016-05-31')]
test = df[(df['created'] >= '2016-06-01') & (df['created'] <= '2016-06-30')]

In [62]:
target = ['price']
feature = ['interest_level',
 'hardwood_floors',
 'doorman',
 'elevator',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'terrace',
 'wheelchair_access',
 'total_rooms','longitude','latitude','pet_friend']

In [63]:
X_train = train[feature]
X_test = test[feature]

In [64]:
from sklearn.linear_model import LinearRegression

In [65]:
model = LinearRegression()

In [66]:
y_train = train[target]
y_test = test[target]

In [67]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [68]:
y_pred_train = model.predict(X_train)

In [69]:
from sklearn.metrics import mean_absolute_error
y_pred_train = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred_train)
print('Train Error: $',mae)

Train Error: $ 737.6158992939207


In [70]:
y_pred_test = model.predict(X_test)
mae_t = mean_absolute_error(y_test, y_pred_test)
print('Test Error: $',mae_t)

Test Error: $ 745.9357573248213


In [71]:
print('Intercept:', model.intercept_,'Coefficients: ', model.coef_)

Intercept: [-1203216.6766591] Coefficients:  [[  -153.70016457   -244.04737449    585.03737501    228.88692557
    -217.15217419   -377.50818516    173.2252827     580.54579379
    -195.7966637    -140.9815332     334.92188739   -434.21635299
     -20.01328631    234.15830783    263.03469437    803.34544749
  -15360.25328873   1674.85025316     45.93665877]]


In [72]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [73]:
print('Train Absolute Error: $',mae)
print('Test Absolute Error: $',mae_t)
print('Mean Square error (Train):',mse_train)
print('Root Mean Square error (Train):',np.sqrt(mse_train))
print('Mean Square error (Test):',mse_test)
print('Root Mean Square error (Test):',np.sqrt(mse_test))
print('R^2 scores (Train):',r2_train)
print('R^2 scores (Test):',r2_test)

Train Absolute Error: $ 737.6158992939207
Test Absolute Error: $ 745.9357573248213
Mean Square error (Train): 1501409.5561320838
Root Mean Square error (Train): 1225.3201851483896
Mean Square error (Test): 1467732.490256296
Root Mean Square error (Test): 1211.5000991565357
R^2 scores (Train): 0.5172085955036927
R^2 scores (Test): 0.5280651584879623
